## 1. Load Libraries

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import warnings
warnings.filterwarnings('ignore')

In [47]:
train_path = 'drive/MyDrive/Colab Notebooks/ML100Days/data/train'
test_path = 'drive/MyDrive/Colab Notebooks/ML100Days/data/test'

print(os.listdir(train_path))

['tulip', 'sunflower', 'rose', 'dandelion', 'daisy']


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Data Preprocessing

In [49]:
IMAGE_SIZE = (240, 320)
BATCH_SIZE = 32

flower_mapping = {'daisy':0, 'dandelion':1, 'rose':2,
                  'sunflower':3, 'tulip':4}
flowers = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [50]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

In [51]:
# 設定輸入的 train / test 資料流
# Generate batches of tensor image data with real-time data augmentation
datagen = ImageDataGenerator(
    rotation_range=10, #隨機旋轉角度的上限 (角度值 : 0 ~ 180)
    width_shift_range=0.2, # 左右隨機平移上限
    height_shift_range=0.2, # 上下隨機平移上限
    shear_range=0.2, # 逆時針的剪切變形角度
    zoom_range=0.2, # 影像倍率放大縮小的上限
    horizontal_flip=True, # 是否左右翻轉
    fill_mode='nearest', # 填補周圍空缺方法
    validation_split=0.2, # 測試集採樣比
    preprocessing_function=preprocess_input) # 資料前處理

In [52]:
train_generator = datagen.flow_from_directory(train_path, target_size=IMAGE_SIZE,
                                              classes=flowers, batch_size=BATCH_SIZE, subset='training') # 設為訓練集
valid_generator = datagen.flow_from_directory(train_path, target_size=(240, 320), classes=flowers,
                                                    batch_size=BATCH_SIZE, subset='validation') # 設為驗證集

Found 2260 images belonging to 5 classes.
Found 563 images belonging to 5 classes.


## 3. Model Setting

In [53]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [54]:
# 使用 Keras Sequential 定義一個 CNN 模型
resnet = ResNet50(input_shape=(240, 320, 3), weights='imagenet', pooling='avg', include_top=False)
output = tf.keras.layers.Dense(5, activation="softmax")(resnet.output)
model = Model(inputs=[resnet.input], outputs=[output])

94765736/94765736 [==============================] - 0s 0us/step


In [55]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 240, 320, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 246, 326, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 120, 160, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 120, 160, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [59]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy',
             metrics=['accuracy'])

In [60]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30)
model_checkpoint = ModelCheckpoint("drive/MyDrive/Colab Notebooks/ML100Days/best_model.h5", save_best_only=True)

In [61]:
history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples // BATCH_SIZE,
                              epochs=10,
                              validation_data=valid_generator, validation_steps=valid_generator.samples // BATCH_SIZE,
                              callbacks=[early_stopping, model_checkpoint])

Epoch 1/10
70/70 [==============================] - 119s 1s/step - loss: 0.0558 - accuracy: 0.9892 - val_loss: 0.2187 - val_accuracy: 0.9375
Epoch 2/10
70/70 [==============================] - 75s 1s/step - loss: 0.0454 - accuracy: 0.9874 - val_loss: 0.2489 - val_accuracy: 0.9265
Epoch 3/10
70/70 [==============================] - 93s 1s/step - loss: 0.0418 - accuracy: 0.9883 - val_loss: 0.2040 - val_accuracy: 0.9357
Epoch 4/10
70/70 [==============================] - 77s 1s/step - loss: 0.0323 - accuracy: 0.9937 - val_loss: 0.2153 - val_accuracy: 0.9393
Epoch 5/10
70/70 [==============================] - 83s 1s/step - loss: 0.0213 - accuracy: 0.9973 - val_loss: 0.2264 - val_accuracy: 0.9320
Epoch 6/10
70/70 [==============================] - 80s 1s/step - loss: 0.0235 - accuracy: 0.9951 - val_loss: 0.2206 - val_accuracy: 0.9393
Epoch 7/10
70/70 [==============================] - 81s 1s/step - loss: 0.0163 - accuracy: 0.9973 - val_loss: 0.2349 - val_accuracy: 0.9393
Epoch 8/10
70/70 [=

## 4. Prediction

In [62]:
best_model = tf.keras.models.load_model("drive/MyDrive/Colab Notebooks/ML100Days/best_model.h5")

In [63]:
import cv2
import glob
from PIL import Image

In [64]:
data = np.array([cv2.resize(np.array(cv2.imread(x)[:,:,::-1]), (320,240)) for x in sorted(glob.glob('drive/MyDrive/Colab Notebooks/ML100Days/data/test/*.jpg'))])
data = preprocess_input(data)

In [65]:
y_pred = best_model.predict(data)

63/63 [==============================] - 11s 159ms/step


In [66]:
submit_id = []
for item in glob.iglob("drive/MyDrive/Colab Notebooks/ML100Days/data/test/*.jpg"):
    submit_id.append(item.removeprefix('drive/MyDrive/Colab Notebooks/ML100Days/data/test/').replace('.jpg',''))

In [70]:
sub = pd.DataFrame({'id': submit_id, 'flower_class': np.argmax(y_pred, 1)})
sub.to_csv('drive/MyDrive/Colab Notebooks/ML100Days/resnet50_output.csv', index=False)